In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import ast
import gensim
import re
from collections import defaultdict

In [43]:
df_all= pd.read_csv('All_data.csv')

In [44]:
df_all

,clean_title,2_way_label,word_count,dic_words
0,differ order,1,2,"{'differ': 1, 'order': 1}"
1,concern sink tini hat,0,4,"{'concern': 1, 'sink': 1, 'tini': 1, 'hat': 1}"
2,leak ambassador,1,2,"{'leak': 1, 'ambassador': 1}"
3,puppi take view,1,3,"{'puppi': 1, 'take': 1, 'view': 1}"
4,face sheet music,0,3,"{'face': 1, 'sheet': 1, 'music': 1}"
...,...,...,...,...
534209,bicycl taxi,1,2,"{'bicycl': 1, 'taxi': 1}"
534210,trump win hous,1,3,"{'trump': 1, 'win': 1, 'hous': 1}"
534211,napoleon island,0,2,"{'napoleon': 1, 'island': 1}"
534212,deep dancer,0,2,"{'deep': 1, 'dancer': 1}"


In [45]:
df_all['Label']=df_all['2_way_label']
df_all['clean_title']=df_all['clean_title'].apply(lambda x: x.split())

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_all.clean_title, df_all.Label, test_size = 0.25, random_state = 0)

# DOC2VEC

In [93]:
def label_sent(corpus, lb_type):
    labeled=[]
    for i,v in enumerate(corpus):
        lb1= lb_type+'_'+str(i)
        labeled.append (gensim.models.doc2vec.TaggedDocument(v, [lb1]))
    return labeled
        
X_train=label_sent(X_train, 'Train')
X_test=label_sent(X_test, 'Test')
X_train[0]

TaggedDocument(words=['paint', 'li', 'south', 'farm'], tags=['Train_0'])

In [94]:
all_data= X_train+ X_test
X_train[0]

TaggedDocument(words=['paint', 'li', 'south', 'farm'], tags=['Train_0'])

In [95]:
from sklearn import utils
model = gensim.models.doc2vec.Doc2Vec(dm=0, vector_size=300, negative=5,  min_count=1, alpha=0.065, min_alpha=0.065)
model.build_vocab([x for x in all_data])
for epoch in range(30):
    model.train(utils.shuffle([x for x in all_data]), total_examples= len(all_data), epochs=1)
    model.alpha-=0.002
    model.min_alpha=model.alpha

In [96]:
def get_vectors(m, corp_sz, v_sz, v_type):
    vectors= np.zeros((corp_sz,v_sz))
    for i in range(0, corp_sz):
        lb1= v_type + '_' + str(i)
        vectors[i]=m.docvecs[lb1]
    return vectors

In [97]:
train_vecs=get_vectors(model, len(X_train), 300, 'Train')
test_vecs=get_vectors(model, len(X_test), 300, 'Test')

C:\Users\ABDELR~1\AppData\Local\Temp/ipykernel_25184/3487055183.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  vectors[i]=m.docvecs[lb1]


In [98]:
train_vecs.shape

(400660, 300)

# Logistic Regression

In [99]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
clf=LogisticRegression(C=1e5)
clf=clf.fit(train_vecs,y_train)
yp=clf.predict(test_vecs)
yp

array([0, 1, 1, ..., 0, 0, 1], dtype=int64)

In [100]:
print(classification_report(y_test,yp))

              precision    recall  f1-score   support

           0       0.68      0.74      0.71     70538
           1       0.68      0.62      0.65     63016

    accuracy                           0.68    133554
   macro avg       0.68      0.68      0.68    133554
weighted avg       0.68      0.68      0.68    133554



In [101]:
from sklearn.linear_model import SGDClassifier
clf=SGDClassifier(max_iter=100000)
clf= clf.fit(train_vecs,y_train)
yp=clf.predict(test_vecs)
print(classification_report(y_test,yp))

              precision    recall  f1-score   support

           0       0.68      0.74      0.71     70538
           1       0.68      0.62      0.65     63016

    accuracy                           0.68    133554
   macro avg       0.68      0.68      0.68    133554
weighted avg       0.68      0.68      0.68    133554



In [102]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
print("GAUSSIAN NB")
naive_bayes_classifier = GaussianNB()
naive_bayes_classifier= naive_bayes_classifier.fit(train_vecs, y_train)
y_pred = naive_bayes_classifier.predict(test_vecs)
print(classification_report(y_test,y_pred))

print("BERNOULLI NB")
bnb = BernoulliNB()
bnb.fit(train_vecs,y_train)
test_ber_doc2vec = bnb.predict(test_vecs)
print(classification_report(y_test,test_ber_doc2vec))

GAUSSIAN NB
              precision    recall  f1-score   support

           0       0.62      0.71      0.67     70538
           1       0.62      0.52      0.56     63016

    accuracy                           0.62    133554
   macro avg       0.62      0.62      0.62    133554
weighted avg       0.62      0.62      0.62    133554

BERNOULLI NB
              precision    recall  f1-score   support

           0       0.69      0.66      0.67     70538
           1       0.63      0.66      0.65     63016

    accuracy                           0.66    133554
   macro avg       0.66      0.66      0.66    133554
weighted avg       0.66      0.66      0.66    133554



In [103]:
from sklearn.linear_model import Perceptron
Per_clf = Perceptron(tol=1e-3, random_state=0)
Per_clf.fit(train_vecs,y_train)
test_Per_doc2vec=Per_clf.predict(test_vecs)
print(classification_report(y_test.values,test_Per_doc2vec))

              precision    recall  f1-score   support

           0       0.65      0.56      0.60     70538
           1       0.58      0.66      0.62     63016

    accuracy                           0.61    133554
   macro avg       0.61      0.61      0.61    133554
weighted avg       0.62      0.61      0.61    133554



In [104]:
from sklearn.neural_network import MLPClassifier
NN = MLPClassifier(random_state=1, max_iter=5000)
NN.fit(train_vecs,y_train.values)
test_NN_doc2vec = NN.predict(test_vecs)
print(classification_report(y_test.values,test_NN_doc2vec))

              precision    recall  f1-score   support

           0       0.69      0.79      0.74     70538
           1       0.72      0.61      0.66     63016

    accuracy                           0.70    133554
   macro avg       0.71      0.70      0.70    133554
weighted avg       0.71      0.70      0.70    133554



In [105]:
from sklearn.tree import DecisionTreeClassifier
clf_decision_doc2vec = DecisionTreeClassifier()
clf_decision_doc2vec.fit(train_vecs,y_train)
test_predictions_doc2vec = clf_decision_doc2vec.predict(test_vecs)
print(classification_report(y_test,test_predictions_doc2vec))

              precision    recall  f1-score   support

           0       0.62      0.62      0.62     70538
           1       0.57      0.57      0.57     63016

    accuracy                           0.60    133554
   macro avg       0.59      0.59      0.59    133554
weighted avg       0.60      0.60      0.60    133554



In [106]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(train_vecs,y_train)
test_predictions_w2vec = clf.predict(test_vecs)
print(classification_report(y_test,test_predictions_w2vec))

              precision    recall  f1-score   support

           0       0.68      0.75      0.71     70538
           1       0.68      0.60      0.64     63016

    accuracy                           0.68    133554
   macro avg       0.68      0.68      0.68    133554
weighted avg       0.68      0.68      0.68    133554



# WORD2VEC

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_all.clean_title, df_all.Label, test_size = 0.25, random_state = 42)

In [66]:
X_train.shape

(400660,)

In [67]:
from gensim.models import Word2Vec
titles_corpus= []
for title in df_all["dic_words"][:]:
    new_list = list(ast.literal_eval(title).keys())
    if (new_list != []):
        titles_corpus.append(new_list)

In [68]:
titles_corpus

[['differ', 'order'],
 ['concern', 'sink', 'tini', 'hat'],
 ['leak', 'ambassador'],
 ['puppi', 'take', 'view'],
 ['face', 'sheet', 'music'],
 ['bride', 'exchang', 'fatal', 'shoot', 'we', 'd'],
 ['meat', 'pig', 'eat', 'milk'],
 ['convert', 'local', 'teen', 'circa', 'ad'],
 ['victori', 'great', 'crusad'],
 ['villag', 'leader', 'clean', 'patch'],
 ['ai', 'fli', 'moon'],
 ['seven', 'sloth'],
 ['babi', 'cow', 'see', 'head', 'day', 'enjoy'],
 ['girl', 'wear', 'virtual', 'realiti', 'hold'],
 ['way', 'rain', 'silhouett', 'bike'],
 ['mental', 'ill', 'easi'],
 ['love', 'peac', 'way', 'soviet', 'union'],
 ['view', 'bar', 'palac'],
 ['goe', 'well', 'red', 'white', 'rememb'],
 ['happi', 'boat'],
 ['tree', 'angri', 'fall'],
 ['governor', 'statement', 'interview'],
 ['join', 'communist', 'parti', 'music', 'video', 'w'],
 ['neighbor', 'ambul', 'compani', 'move', 'get'],
 ['creepi', 'guy', 'hair', 'stare'],
 ['rain', 'iron', 'wall', 'beauti', 'red'],
 ['presid', 'make', 'point', 'parliament'],
 ['final

In [69]:
modell = gensim.models.Word2Vec(titles_corpus, min_count = 1, vector_size= 500)
modell.save('word2vec_model_file.model')

# Training (Log, Naive, DecT, SVM, Percep, Neural Netw) 


In [70]:
train_data= pd.DataFrame()
train_data['Word_list']= X_train
idx = X_train.index[0]
with open('train_word2vec.csv', 'w+') as word2vec_file:
    for index, row in train_data.iterrows():
        model_vector = (np.mean([modell.wv[token] for token in row['Word_list']], axis=0)).tolist()
        if index == idx:
            header = ",".join(str(ele) for ele in range(500))
            word2vec_file.write(header)
            word2vec_file.write("\n")

        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(500)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [71]:
w2f_df = pd.read_csv('train_word2vec.csv')
w2f_df.values.shape

(400660, 500)

In [72]:
clf=LogisticRegression(C=1e5,solver='lbfgs', max_iter=5000)
clf=clf.fit(w2f_df.values,y_train.values)

In [73]:
test_data= pd.DataFrame()
test_data['Word_list']= X_test
idx = X_test.index[0]
with open('test_word2vec.csv', 'w+') as word2vec_file:
    for index, row in test_data.iterrows():
        model_vector = (np.mean([modell.wv[token] for token in row['Word_list']], axis=0)).tolist()
        if index == idx:
            header = ",".join(str(ele) for ele in range(500))
            word2vec_file.write(header)
            word2vec_file.write("\n")

        if type(model_vector) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
        else:
            line1 = ",".join([str(0) for i in range(500)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [74]:
w2f_test_df = pd.read_csv('test_word2vec.csv')
w2f_test_df.shape

(133554, 500)

In [85]:
modell.wv.most_similar("book")

[('newspap', 0.6760290265083313),
 ('copi', 0.6708730459213257),
 ('textbook', 0.6647543907165527),
 ('page', 0.6197590231895447),
 ('card', 0.6169708967208862),
 ('letter', 0.5677825212478638),
 ('text', 0.5625329613685608),
 ('album', 0.5575341582298279),
 ('calendar', 0.5557220578193665),
 ('error', 0.5418707728385925)]

In [86]:
test_predictions_w2vec = clf.predict(w2f_test_df.values)
print(classification_report(y_test.values,test_predictions_w2vec))

              precision    recall  f1-score   support

           0       0.71      0.71      0.71     70421
           1       0.68      0.68      0.68     63133

    accuracy                           0.70    133554
   macro avg       0.69      0.70      0.69    133554
weighted avg       0.70      0.70      0.70    133554



In [87]:
from sklearn.tree import DecisionTreeClassifier
clf_decision_word2vec = DecisionTreeClassifier()
clf_decision_word2vec.fit(w2f_df.values,y_train.values)
test_decision_word2vec = clf_decision_word2vec.predict(w2f_test_df.values)
print(classification_report(y_test.values,test_decision_word2vec))

              precision    recall  f1-score   support

           0       0.67      0.71      0.69     70421
           1       0.65      0.61      0.63     63133

    accuracy                           0.66    133554
   macro avg       0.66      0.66      0.66    133554
weighted avg       0.66      0.66      0.66    133554



In [88]:
from sklearn.linear_model import Perceptron
Per_clf = Perceptron(tol=1e-3, random_state=0)
Per_clf.fit(w2f_df.values,y_train.values)
test_Per_word2vec=Per_clf.predict(w2f_test_df.values)
print(classification_report(y_test.values,test_Per_word2vec))

              precision    recall  f1-score   support

           0       0.63      0.64      0.63     70421
           1       0.59      0.57      0.58     63133

    accuracy                           0.61    133554
   macro avg       0.61      0.61      0.61    133554
weighted avg       0.61      0.61      0.61    133554



In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.base import clone
skfolds = StratifiedKFold(n_splits=100, shuffle=True, random_state=42)
X=w2f_df
Y=y_train
NN = MLPClassifier(random_state=1, max_iter=5000, activation='logistic')
all_y=[]
all_predicted=[]
for train_index, test_index in skfolds.split(X, Y):
    X_train_folds = X.iloc[train_index]
    y_train_folds = Y.iloc[train_index]
    X_test_fold = X.iloc[test_index]
    y_test_fold = Y.iloc[test_index]
    NN.fit(X_train_folds, y_train_folds)
    y_pred= NN.predict(X_test_fold)
    for i in y_pred:
        all_predicted.append(int(i))
    for j in y_test_fold:
        all_y.append(j)

print(classification_report(all_y, all_predicted))

In [89]:
from sklearn.neural_network import MLPClassifier
NN = MLPClassifier(random_state=1, max_iter=5000, activation='logistic')
NN.fit(w2f_df.values,y_train.values)
test_NN_word2vec = NN.predict(w2f_test_df.values)
print(classification_report(y_test.values,test_NN_word2vec))

              precision    recall  f1-score   support

           0       0.74      0.75      0.74     70421
           1       0.72      0.70      0.71     63133

    accuracy                           0.73    133554
   macro avg       0.73      0.73      0.73    133554
weighted avg       0.73      0.73      0.73    133554



In [90]:
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
gnb = GaussianNB()
gnb.fit(w2f_df.values,y_train.values)
test_gnb_word2vec = gnb.predict(w2f_test_df.values)
print(classification_report(y_test.values,test_gnb_word2vec))

bnb = BernoulliNB()
bnb.fit(w2f_df.values,y_train.values)
test_ber_word2vec = gnb.predict(w2f_test_df.values)
print(classification_report(y_test.values,test_ber_word2vec))

              precision    recall  f1-score   support

           0       0.72      0.53      0.61     70421
           1       0.59      0.77      0.67     63133

    accuracy                           0.64    133554
   macro avg       0.65      0.65      0.64    133554
weighted avg       0.66      0.64      0.64    133554

              precision    recall  f1-score   support

           0       0.72      0.53      0.61     70421
           1       0.59      0.77      0.67     63133

    accuracy                           0.64    133554
   macro avg       0.65      0.65      0.64    133554
weighted avg       0.66      0.64      0.64    133554



In [91]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(w2f_df,y_train)
test_predictions_w2vec = clf.predict(w2f_test_df)
print(classification_report(y_test.values,test_predictions_w2vec))

              precision    recall  f1-score   support

           0       0.71      0.70      0.70     70421
           1       0.67      0.68      0.67     63133

    accuracy                           0.69    133554
   macro avg       0.69      0.69      0.69    133554
weighted avg       0.69      0.69      0.69    133554

